In [3]:
import os
import re
import ast
from tqdm import tqdm
import nltk
import re

In [32]:
movie_conversation_fields = ['char1ID', 'char2ID', 'movieID', 'utter']

In [127]:
def getconv():
    lines = getlines()
    with open('movie_conversations.txt', 'r') as f:
        conversations = []
        for line in f:
            values = line.split(' +++$+++ ')
            #print(len(values))
            lineobj = {}
            for i, field in enumerate(movie_conversation_fields):
                lineobj[field] = values[i]
            lineIds = ast.literal_eval(lineobj["utter"])
            lineobj['lines'] = []
            for lineid in lineIds:
                lineobj['lines'].append(lines[lineid]['text'])
            conversations.append(lineobj)
    return(conversations)

In [128]:
conversations = getconv()

In [129]:
conversations[0]

{'char1ID': 'u0',
 'char2ID': 'u2',
 'movieID': 'm0',
 'utter': "['L194', 'L195', 'L196', 'L197']\n",
 'lines': ['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n',
  "Well, I thought we'd start with pronunciation, if that's okay with you.\n",
  'Not the hacking and gagging and spitting part.  Please.\n',
  "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"]}

In [48]:
movie_lines_fields = ['lineId', 'charId', 'movieId', 'charname', 'text']

In [82]:
def getlines():
    with open('movie_lines.txt', 'r', encoding='iso-8859-1') as f:
        lines = {}
        for line in f:
            values = line.split(' +++$+++ ')
            lineobj = {}
            for i, field in enumerate(movie_lines_fields):
                lineobj[field] = values[i]
                #lineobj['id'] = int(re.sub('L','',lineobj['lineId']))
            lines[lineobj['lineId']] = lineobj
    return(lines)

In [120]:
lines = getlines()

In [121]:
lines

{'L1045': {'lineId': 'L1045',
  'charId': 'u0',
  'movieId': 'm0',
  'charname': 'BIANCA',
  'text': 'They do not!\n'},
 'L1044': {'lineId': 'L1044',
  'charId': 'u2',
  'movieId': 'm0',
  'charname': 'CAMERON',
  'text': 'They do to!\n'},
 'L985': {'lineId': 'L985',
  'charId': 'u0',
  'movieId': 'm0',
  'charname': 'BIANCA',
  'text': 'I hope so.\n'},
 'L984': {'lineId': 'L984',
  'charId': 'u2',
  'movieId': 'm0',
  'charname': 'CAMERON',
  'text': 'She okay?\n'},
 'L925': {'lineId': 'L925',
  'charId': 'u0',
  'movieId': 'm0',
  'charname': 'BIANCA',
  'text': "Let's go.\n"},
 'L924': {'lineId': 'L924',
  'charId': 'u2',
  'movieId': 'm0',
  'charname': 'CAMERON',
  'text': 'Wow\n'},
 'L872': {'lineId': 'L872',
  'charId': 'u0',
  'movieId': 'm0',
  'charname': 'BIANCA',
  'text': "Okay -- you're gonna need to learn how to lie.\n"},
 'L871': {'lineId': 'L871',
  'charId': 'u2',
  'movieId': 'm0',
  'charname': 'CAMERON',
  'text': 'No\n'},
 'L870': {'lineId': 'L870',
  'charId': 'u

In [76]:
sorted_lines = sorted(lines, key = lambda i: i['id'])

In [134]:
def extractText(line, fast_preprocessing=True):
    if fast_preprocessing:
        GOOD_SYMBOLS_RE = re.compile('[^0-9a-z ]')
        REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#+_]')
        REPLACE_SEVERAL_SPACES = re.compile('\s+')

        line = line.lower()
        line = REPLACE_BY_SPACE_RE.sub(' ', line)
        line = GOOD_SYMBOLS_RE.sub('', line)
        line = REPLACE_SEVERAL_SPACES.sub(' ', line)
        return line.strip()
    else:
        return nltk.word_tokenize(line)

def splitConversations(conversations, max_len=20, fast_preprocessing=True):
    data = []
    for i, conversation in enumerate(tqdm(conversations)):
        lines = conversation['lines']
        for i in range(len(lines) - 1):
            request = lines[i]
            reply = lines[i + 1]
            if 0 < len(request) <= max_len and 0 < len(reply) <= max_len:
                data += [(request, reply)]
    return data

In [97]:
for conversation in conversations:
    conversation['dialogues'] = []
    clines = conversation['lines']
    for i in range(len(clines)):
        conversation['dialogues'].append(lines[clines[i]]['text'])

In [101]:
conversations[2]

{'char1ID': 'u0',
 'char2ID': 'u2',
 'movieID': 'm0',
 'utter': "['L200', 'L201', 'L202', 'L203']\n",
 'lines': ['L200', 'L201', 'L202', 'L203'],
 'dialogues': ["No, no, it's my fault -- we didn't have a proper introduction ---\n",
  'Cameron.\n',
  "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n",
  'Seems like she could get a date easy enough...\n']}

In [99]:
sorted_lines[84:89]

[{'lineId': 'L194',
  'id': 194,
  'charId': 'u0',
  'movieId': 'm0',
  'charname': 'BIANCA',
  'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
 {'lineId': 'L195',
  'id': 195,
  'charId': 'u2',
  'movieId': 'm0',
  'charname': 'CAMERON',
  'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
 {'lineId': 'L196',
  'id': 196,
  'charId': 'u0',
  'movieId': 'm0',
  'charname': 'BIANCA',
  'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
 {'lineId': 'L197',
  'id': 197,
  'charId': 'u2',
  'movieId': 'm0',
  'charname': 'CAMERON',
  'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"},
 {'lineId': 'L198',
  'id': 198,
  'charId': 'u0',
  'movieId': 'm0',
  'charname': 'BIANCA',
  'text': "You're asking me out.  That's so cute. What's your name again?\n"}]

In [118]:
import dialogues

In [119]:
data = dialogues.readCornellData('/Users/rohan/Desktop/projects/python-chatbot')

100%|██████████| 83097/83097 [00:03<00:00, 24476.13it/s]


In [116]:
len(data)

24792

In [117]:
data

[('there', 'where'),
 ('have fun tonight', 'tons'),
 ('what good stuff', 'the real you'),
 ('wow', 'lets go'),
 ('she okay', 'i hope so'),
 ('they do to', 'they do not'),
 ('who', 'joey'),
 ('its more', 'expensive'),
 ('hey sweet cheeks', 'hi joey'),
 ('whereve you been', 'nowhere hi daddy'),
 ('what', 'in 9th for a month'),
 ('in 9th for a month', 'why'),
 ('let go', 'you set me up'),
 ('you set me up', 'i just wanted'),
 ('patrick is that a', 'perm'),
 ('daddy no', 'just for a minute'),
 ('you the new guy', 'so they tell me'),
 ('thirtytwo', 'get out'),
 ('her favorite uncle', 'dead at fortyone'),
 ('hes pretty', 'okay i wasnt sure'),
 ('sure', 'then go get her'),
 ('she kissed me', 'where'),
 ('whats the worst', 'you get the girl'),
 ('hey do you mind', 'not at all'),
 ('where ya goin', 'away'),
 ('away', 'your sister here'),
 ('yeah', 'what do you think'),
 ('its about time', 'a deals a deal'),
 ('howd you do it', 'do what'),
 ('hey', 'are you lost'),
 ('he always look so', 'block 

In [135]:
data = splitConversations(conversations)

100%|██████████| 83097/83097 [00:00<00:00, 682369.91it/s]


In [137]:
data

[('There.\n', 'Where?\n'),
 ('Have fun tonight?\n', 'Tons\n'),
 ('What good stuff?\n', 'The "real you".\n'),
 ('Wow\n', "Let's go.\n"),
 ('She okay?\n', 'I hope so.\n'),
 ('They do to!\n', 'They do not!\n'),
 ('Who?\n', 'Joey.\n'),
 ("It's more\n", 'Expensive?\n'),
 ('Hey, sweet cheeks.\n', 'Hi, Joey.\n'),
 ('Let go!\n', 'You set me up.\n'),
 ('You set me up.\n', 'I just wanted --\n'),
 ('Daddy, no!\n', 'Just for a minute\n'),
 ('You the new guy?\n', 'So they tell me...\n'),
 ('Thirty-two.\n', 'Get out!\n'),
 ('Her favorite uncle\n', 'Dead at forty-one.\n'),
 ('Sure\n', 'Then, go get her\n'),
 ('She kissed me.\n', 'Where?\n'),
 ("What's the worst?\n", 'You get the girl.\n'),
 ('Hey -- do you mind?\n', 'Not at all\n'),
 ('Where ya goin?\n', 'Away.\n'),
 ('Away.\n', 'Your sister here?\n'),
 ('Yeah\n', 'What do you think?\n'),
 ("It's about time.\n", "A deal's a deal.\n"),
 ("How'd you do it?\n", 'Do what?\n'),
 ('Hey.\n', 'Are you lost?\n'),
 ('He always look so\n', 'Block E?\n'),
 ("Wha

In [36]:
import xml.etree.ElementTree as ET
import datetime
import os
import sys
import json
import re
import pprint
import ast
import nltk
import tqdm
from time import time

In [11]:
open_subtitles_path = "/Users/rohan/Desktop/projects/python-chatbot/OpenSubtitles/xml/en/"

In [30]:
files = os.listdir(open_subtitles_path)
xml_files = []
for i in files:
    try:
        filelist = os.listdir(open_subtitles_path+i)
        for file in filelist:
            try:
                finallist = os.listdir(open_subtitles_path+i+"/"+file)
                for x in finallist:
                    if x.endswith('.xml'):
                        xml_files.append(open_subtitles_path+i+"/"+file+"/"+x)
            except NotADirectoryError:
                if file.endswith('.xml'):
                    xml_files.append(open_subtitles_path+i+"/"+file)
    except NotADirectoryError:
        if file.endswith('.xml'):
                xml_files.append(open_subtitles_path+i)

In [50]:
def genList(tree):
    root = tree.getroot()
    timeFormat = '%H:%M:%S'
    maxDelta = datetime.timedelta(seconds=1)
    startTime = datetime.datetime.min
    strbuf = ''
    sentList = []
    for child in root:
        for elem in child:
            if elem.tag == 'time':
                try:
                    elemID = elem.attrib['id']
                    elemVal = elem.attrib['value'][:-4]
                    if elemID[-1] == 'S':
                        startTime = datetime.datetime.strptime(elemVal, timeFormat)
                    else:
                        sentList.append((strbuf.strip(), startTime, datetime.datetime.strptime(elemVal, timeFormat)))
                        strbuf = ''
                except:
                    continue
            else:
                try:
                    strbuf = strbuf + " " + elem.text
                except:
                    pass
    conversations = []
    for idx in range(0, len(sentList) - 1):
        cur = sentList[idx]
        nxt = sentList[idx + 1]
        if nxt[1] - cur[2] <= maxDelta and cur and nxt:
            tmp = {}
            tmp["lines"] = []
            tmp["lines"].append(getLine(cur[0]))
            tmp["lines"].append(getLine(nxt[0]))
            if filterqa(tmp):
                conversations.append(tmp)
    return conversations

def getLine(sentence):
    tag_re = re.compile(r'(<!--.*?-->|<[^>]*>)')
    line = {}
    line["text"] = tag_re.sub('', sentence).replace('\\\'','\'').strip().lower()
    return line


def filterqa(lines):
    # Use the followint to customize filtering of QA pairs
    startwords = ("what", "how", "when", "why", "where", "do", "did", "is", "are", "can", "could", "would", "will")
    question = lines["lines"][0]["text"]
    if not question.endswith('?'):
        return False
    if not question.split(' ')[0] in startwords:
        return False
    return True

In [51]:
len(xml_files)

2317

In [52]:
def getConversation(files):
    converations = []
    for file in files:
        converations.extend(genList(ET.parse(file)))
    return converations

In [53]:
os_conv = getConversation(xml_files)

In [54]:
len(os_conv)

68367

In [66]:
os_conv[2]['lines'][1]['text']

"you' re damn right no ."

In [2]:
import pandas as pd

In [3]:
cornell_data = pd.read_csv("cornell_pairs.csv")

In [72]:
cornell_data['question'][1]

'cesc ma tete this is my head'

In [4]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

In [5]:
model = defaultdict(lambda:defaultdict(lambda:0))

for i,r in cornell_data.iterrows():
    sentence = str(r['question']) +" "+ str(r['answer'])
    for w1,w2,w3 in trigrams(sentence.split(' '), pad_right=True, pad_left=True):
        model[(w1,w2)][w3] += 1
        
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3]/= total_count

In [ ]:
dict(model["her","man"])

In [9]:
import random

# starting words
text = ["how", "is"]
sentence_finished = False
 
while not sentence_finished:
    # select a random probability threshold  
    r = random.random()
    accumulator = .0
    for word in model[tuple(text[-2:])].keys():
        accumulator += model[tuple(text[-2:])][word]
        # select words that are above the probability threshold
        if accumulator >= r:
            text.append(word)
            break
    if text[-2:] == [None, None]:
        sentence_finished = True
print (' '.join([t for t in text if t]))

how is she ? where' s the cocky motherfucker who come on thats ridic


In [7]:
def refine_sentences(text):
    return(re.sub('n\' t',' not',text))

In [8]:
x = "aren\' t"
refine_sentences(x)

'are not'